# Exercise - Oja's rule

In [ ]:
# Install matplotlib for plotting results. Numpy is included in the package
# sklearn will be used for implementing the standard PCA technique
!pip install matplotlib
!pip install sklearn

# Principal Components Analysis (PCA): Simple application

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import sklearn.decomposition

Principal components analysis (PCA) is a well-known technique from multivariate statistics used for simplifying systems formed by many variables. After applying PCA to a system with N variables, it is possible to reduce the system to M principal components (PCs) (where M < N), while maintaining most of the relevant information.

PCA finds the correlations among the different variables and defines new variables - the PCs - which are obtained by linear combinations of the former.

In this exercise, we will create synthetic data for a dummy example with two variables which are highly correlated. Namely, this data corresponds to the model of a simple mechanical spring that follows Hooke's law for elastic materials, i.e., $\Delta l = F * k$. Let's supose that we have two sensors monitoring a spring and providing the elongation and force applied to the spring. Since these two variables are highly correlated, PCA should be able to detect this correlation and reduce the system to a single principal component (PC).


In [3]:
k_elastic = 3   # N/m
# Generate the real displacement of the spring during the simulation,
# and the equivalent noisy measurements from the sensor
real_displacement = np.arange(-5, 5, 0.04)
measured_displacement = real_displacement + np.random.normal(0, 0.5, 250)

# Calculate the ideal force and add noise to the measured variable
real_force = k_elastic * real_displacement
measured_force = real_force + np.random.normal(0, 1, real_force.size)

# Normalize the measured values between -1 and 1
measured_displacement /= np.abs(measured_displacement).max()
measured_force /= np.abs(measured_force).max()

Now, we will apply the standard PCA method on this artificially generated data. The PCA should show one principal component carrying most of the information, since the displacement and the force are linearly dependent variables, and the variation is only induced by noise.

In [4]:
# Apply PCA using the traditional method from sklearn
samples = np.vstack((measured_displacement, measured_force)).transpose()
pca = sklearn.decomposition.PCA()
pca = pca.fit(samples)

The variable ```pca``` is an object containing, among others, the eigenvectors and eigenvalues corresponding to the different PCs. Let's visualize now the original data, and the direction of the two principal components that the previous method found

In [ ]:
# Generate a square figure, so the orthonormality can be directly observed 
fig,ax = plt.subplots(1, figsize=(10,10))
eigenvectors = pca.components_
eigenvalues = pca.explained_variance_
v_1 = eigenvectors[0]
v_2 = eigenvectors[1]
d_0 = measured_displacement.mean()
f_0 = measured_force.mean()
plt.scatter(measured_displacement, measured_force)
# Print in red the first PC, and in green the second PC
plt.arrow(d_0, f_0, v_1[0], v_1[1], color="red", width=0.01)
plt.arrow(d_0, f_0, v_2[0], v_2[1], color="green", width=0.01)
plt.ylim(-2, 2)
plt.xlim(-2, 2)

ax.set_yticklabels([])
ax.set_xticklabels([])
ax.set_xlabel("Displacement", fontsize=20)
ax.set_ylabel("Force", fontsize=20)

plt.show()

Let's also see the explained variance by each PC. It is calculated as the relative importance of each corresponding eigenvalue relative to the total sum of all eigenvalues. It can be easily seen that most of the information in the input data is contained in the first PC.

In [ ]:
for i, ratio in enumerate(pca.explained_variance_ratio_):
  print("PC {}: {}".format(i+1, ratio))

In this case the usage of PCA is redundant because we have full knowledge of the system and we could take just one of the two variables manually. However, we often have to deal with problems that involve many variables, which are probably correlated with relationships that we are not aware of.

Some more complicated examples where PCA is a very powerful tool are the following:
* Summarizing the relevant features that define a human face from an image, and develop a classifier that labels the name of the corresponding person (always respecting the data privacy of the face owners). The PCs of a face image are typically known as _eigenfaces_.
* Analyzing the variables from thousands of companies in the stock market, and provide a quantitative measurement of how risky is to invest in a specific sector
* Reduce all physiological measures of certain animals into a single score that reflects certain features, e.g., provide an educated guess of the age of a cow by providing the weight, height, hoofs size, hornes length, etc.
* Compression of very large datasets

# Principal components analysis with Oja's rule

Let's now apply Oja's rule to the previous spring example. The task is to obtain the eigenvector that corresponds to the first PC of the data, since the system can be described with just the main PC.

Let's create a class that allows us to simulate Oja's rule over all the input samples. After the class iterates over all samples, this small neural network should have learned the components of the first PC, and store it in the weights of the network.

Oja's rule is a form of unsupervised, Hebbian learning that typically works with data encoded in the form of rates. Therefore, the inputs to our class represent the rates of the network connections. This rule is elegant and simple, as it provides competition and saturation to the calculated weights. On the negavite side, it is hard to think of biological neurons implementing this rule, since that would require them to compute the covariance matrix of all inputs, even before they go through the synaptic weights.

In our case, our network is composed by two inputs (force and displacement), and one output (the main principal component)

In [7]:
class OjaNetwork():
    """
    Class containing a model of the Oja's learning rule

    The class iterates over the provided samples and learns the weights,
    which ideally converge to the eigenvector representing the main
    principal component of the distribution.
    """
    def __init__(self, dynamic_learning=False, dims=2):
        # With dynamic learning, the learning rate will slowly decrease over time
        self.dynamic_learning = dynamic_learning
        # Initial learning rate (stays the same if dynamic learning is False)
        self.alpha = 0.5
        self.t = 0
        # Time step between iterations
        self.delta_t = 1
        # Initialize the weights with random values around 0.1
        self.weights = (np.random.rand(dims)/10).reshape((2,1))
        self.weights_trace = np.copy(self.weights)

    def update_learning_rate(self):
        """
        Update the learning rate of the weight update
        """
        self.alpha = 0.25 / np.sqrt(self.t)
        return self.alpha

    def update_weights(self, X):
        """
        Update the weights according to Oja's learning rule
        """
        # Oja's rule simple form: delta_w = alpha * y * (X - y*W)
        y = np.dot(X.transpose(), self.weights)
        delta_w = self.alpha * y * (X-y*self.weights) * self.delta_t
        # Alternative equation of Oja's rule: delta_w = alpha * (C*w-w^T*C*w*w)
        # cov = np.dot(X, X.transpose())
        # delta_w = self.alpha * (np.dot(cov, self.weights)
        #                         - np.dot(self.weights.transpose(),
        #                                  np.dot(cov, self.weights))
        #                         * self.weights)
        self.weights += delta_w
        return self.weights

    def run(self, X):
        """
        Iterate over all samples and apply Oja's rule on each step
        """
        for sample in X:
            self.t += self.delta_t
            if self.dynamic_learning:
                self.update_learning_rate()
            self.update_weights(sample.reshape((2, 1)))
            self.weights_trace = np.hstack((self.weights_trace, self.weights))
        return

Let's instantiate the previous class and run it with the simulated-spring data create before.

In [8]:
# Apply Oja's rule on the samples and compare with PCA decomposition
oja_network = OjaNetwork(dynamic_learning=False)
oja_network.run(np.vstack((measured_displacement, measured_force)).transpose())

Let's now plot how the weights of the network evolve over the simulation, and compare them with the desired goal, which is the PC obtained with the standard method in the previous section.

In [ ]:
v_1 = pca.components_[0]
fig, axs = plt.subplots(2, figsize=(10, 10))
axs[0].set_title("Oja's rule vs. PCA", fontsize=18)
axs[0].plot(oja_network.weights_trace[0])
# We use the abs function, because the standard method can provide a vector v or
# v' pointing in the opposite direction. The sign is irrelevant in PCA, only the
# direction matters
axs[0].axhline(np.abs(v_1[0]), color="green")

axs[1].plot(oja_network.weights_trace[1])
axs[1].axhline(np.abs(v_1[1]), color="green")
axs[1].set_xlabel("Iteration Nº", fontsize=16)
plt.show()

# Open questions

* What happens if you modify the learning rate (self.alpha) dynamically? You can easily do this by setting ```dynamic_learning``` to True (remember to run the last three blocks of code after)
* What is the impact of the learning rate? Try using higher and smaller values of self.alpha and see how it affects the final outcome. Which changes in the initial weights, initial learning rate, and learning rate function can facilitate a faster convergence to the desired values? At what cost, or which prior information would we need for implementing this changes?
* Imagine that now you would like to obtain more than one principal component. How would you avoid all output neurons converging towards the main PC?